# Top-Down parser
## Általános felülröl lefelé elemzés

_Sipos István_  
_2020.04.13_

### Feladat

Adjunk olyan algoritmust, amely tetszőleges $G=(V_N, V_T , S, H)$ környezetfüggetlen nyelvtan és $w \in \Sigma *$ szó esetén eldönti, hogy $w \in L(G)$ teljesül-e!

#### Algoritmus inputja:
Egy nem balrekurzív $G=(V_N , V_T , S, H)$ környezetfüggetlen nyelvtan és egy $w=a_1, a_2, ... a_n , n > 0$ input szó. A $w$ szót $n+1$. szimbólumként egy # jel zárja le. A # nem tartozik sem $V_N$ -hez, sem  $V_T$-hez.

#### Algoritmus outputja:
Igen jelzés, és a $w$ szónak egy baloldali levezetése, ha $w \in L(G)$. Nem jelzés egyébként.

#### Módszer:
1. Minden $A \in V_N$ esetén rögzítsük le az $A$ alternatíváit $A \rightarrow \gamma_1 \mid \gamma_2 \mid ... \mid \gamma_k$ alakban. Az $A$ i-dik alternatíváját $A_i$ jelöli. (Implementáláskor az (A, i) párt alkalmazzuk $A_i$jelölésére.) 
2. Az elemzés $(s, i, \alpha, \beta)$ alakú konfigurációk sorozata.
3. A konfigurációk halmazán megadunk egy \vdash átmeneti relációt. A rákövetkező konfiguráció meghatározása az alábbiakban megadott felsorolásból történik.
4. A kezdő konfiguráció $(q,1, \lambda, S)$. A befejező konfiguráció: $(t, n+1,\alpha, \lambda)$ $w \in L(G)$ akkor és csak akkor, ha $(q,1, \lambda, S) \vdash* (t, n+1, \alpha, \lambda)$

#### A konfiguráció:

$(s, i, \alpha, \beta)$ értelmezése:  
  * s: az elemzés állapota. 
    * q: normál
    * t: elfogadó
    * b: backtrack
  * i: pointer az input szóban (1 < i < n+1)
  * $\alpha$: jobbvégtetejű verem, az elemzés története backtrack-hez és a baloldali levezetéshez. (Passzív verem)
  * $\beta$: balvégtetejű verem, a még levezetendő baloldali mondatforma. (Aktív verem)

### Implementáció

In [1]:
from IPython.display import display, Math

A konfigurációban tárolt állapotok

In [2]:
class Status():
    def __init__(self, text):
        self.text = text
    def __str__(self):
        return self.text
    def __eq__(self, other):
        return isinstance(other, Status) and self.text == other.text

NORMAL = Status('q')
ACCEPT = Status('t')
BACKTRACK = Status('b')

Az aktív és passzív vermek osztálya

In [3]:
class Stack(list):
    @property
    def empty(self):
        return len(self) == 0
    
    def __str__(self):
        if self.empty:
            return r'\{\lambda\}'
        return r'\{' + r' \mid '.join(str(e) for e in self) + r'\}'

A terminális és nem-terminális típusok

In [4]:
class NonTerminal():
    def __init__(self, non_terminal, index = None):
        if isinstance(non_terminal, NonTerminal):
            self.non_terminal = non_terminal.non_terminal
        else:
            self.non_terminal = non_terminal
        self.index = index
        
    def __str__(self):
        if self.index == None:
            return str(self.non_terminal)
        return r'{}_{}'.format(str(self.non_terminal), self.index)

    def __eq__(self, other):
        return isinstance(other, NonTerminal) and self.non_terminal == other.non_terminal
    
class Terminal():
    def __init__(self, terminal):
        if isinstance(terminal, Terminal):
            self.terminal = terminal.terminal
        else:
            self.terminal = terminal
        
    def __str__(self):
        return str(self.terminal)

    def __eq__(self, other):
        return isinstance(other, Terminal) and self.terminal == other.terminal

Az aktuális konfigurációt tároló osztály

In [5]:
class Configuration():
    def __init__(self, status, index, active_stack, passive_stack):
        self.status = status
        self.index = index
        self.active_stack = active_stack
        self.passive_stack = passive_stack
        
    def __str__(self):
        return r'(' +\
            str(self.status) + ', ' +\
            str(self.index) + ', ' +\
            str(self.active_stack) + ', ' +\
            str(self.passive_stack) +\
            ')'

    @property
    def accepted(self):
        return self.status == ACCEPT and self.passive_stack.empty

Segédfüggvények

In [6]:
LOGGING_ENABLED = True

def log(msg):
    def decorator_log(func):
        def wrapper_log(*args, **argv):
            if LOGGING_ENABLED:
                print(msg)
            return func(*args, **argv)
        return wrapper_log
    return decorator_log

A nyelvet tároló, és a levezethetöséget kiszámító osztály.  
A konstruktor paraméterei:
  * **S**: _NonTerminal_ típusú érték
  * **H**: Szabályok listája. Az egyes szabályok _(NonTerminal, \[(Terminal|NonTerminal), ...\])_ felépítésüek lehetnek

In [7]:
class Lang():
    def __init__(self, S, H):
        self.S = S
        self.H = H

    def substitutes(self, non_terminal):
        return [e for (NT, e) in self.H if NT == non_terminal]

    @log('Kiterjesztés')
    def extend(self, configuration):
        substitute = self.substitutes(configuration.passive_stack[0])[0]
        return Configuration(NORMAL,
                         configuration.index,
                         Stack([*(configuration.active_stack), NonTerminal(configuration.passive_stack[0], 1)]),
                         Stack([*(substitute), *(configuration.passive_stack[1:])])
                         )
    @log('Sikeres input ellenörzés')
    def advance(self, configuration):
        return Configuration(NORMAL,
                         configuration.index + 1,
                         Stack([*(configuration.active_stack), configuration.passive_stack[0]]),
                         Stack(configuration.passive_stack[1:])
                         )

    @log('Sikeres elemzés!')
    def accept(self, configuration):
        return Configuration(ACCEPT,
                         configuration.index,
                         Stack(configuration.active_stack),
                         Stack(configuration.passive_stack)
                         )

    @log('Sikertelen input illesztés')
    def set_backtrack(self, configuration):
        return Configuration(BACKTRACK,
                         configuration.index,
                         Stack(configuration.active_stack),
                         Stack(configuration.passive_stack)
                         )

    @log('Backtrack az inputban')
    def backtrack(self, configuration):
        return Configuration(BACKTRACK,
                         configuration.index - 1,
                         Stack([*(configuration.active_stack[:-1])]),
                         Stack([configuration.active_stack[-1], *(configuration.passive_stack)])
                         )
    
    @log('Backtrack a kiterjesztésben I.')
    def next_substitute(self, configuration):
        non_terminal = configuration.active_stack[-1]
        current_substitute = self.substitutes(non_terminal)[non_terminal.index-1]
        substitute = self.substitutes(non_terminal)[non_terminal.index]
        return Configuration(NORMAL,
                         configuration.index,
                         Stack([*(configuration.active_stack[:-1]), NonTerminal(non_terminal, non_terminal.index+1)]),
                         Stack([*(substitute[::-1]), *(configuration.passive_stack[len(current_substitute):])])
                         )

    @log('Backtrack a kiterjesztésben III.')
    def previous(self, configuration):
        non_terminal = configuration.active_stack[-1]
        substitute = self.substitutes(non_terminal)[non_terminal.index-1]
        
        return Configuration(BACKTRACK,
                         configuration.index,
                         Stack([*(configuration.active_stack[:-1])]),
                         Stack([NonTerminal(configuration.active_stack[-1]), *(configuration.passive_stack[len(substitute):])])
                         )

    def is_deriveable(self, w):
        """A w szó levezetése"""
        
        configuration = Configuration(NORMAL, 1, Stack(), Stack([self.S]))

        while True:
            if LOGGING_ENABLED:
                display(Math(str(configuration)))

            if configuration.accepted:
                return True, configuration.active_stack
            elif configuration.status == NORMAL:
                if configuration.index == len(w) and configuration.passive_stack.empty:
                    configuration = self.accept(configuration)
                elif isinstance(configuration.passive_stack[0], NonTerminal):
                    configuration = self.extend(configuration)
                elif configuration.passive_stack[0] == w[configuration.index -1]:
                    configuration = self.advance(configuration)
                elif configuration.passive_stack[0] != w[configuration.index -1]:
                    configuration = self.set_backtrack(configuration)
                else:
                    assert False
            elif configuration.status == BACKTRACK:
                if isinstance(configuration.active_stack[-1], Terminal):
                    configuration = self.backtrack(configuration)
                elif isinstance(configuration.active_stack[-1], NonTerminal):
                    non_terminal = configuration.active_stack[-1]
                    if non_terminal.index+1 == len(self.substitutes(non_terminal)):
                        configuration = self.next_substitute(configuration)
                    elif configuration.index == 1 and non_terminal == self.S and non_terminal.index == len(self.substitutes(non_terminal)):
                        return False, None
                    else:
                        configuration = self.previous(configuration)
                else:
                    assert False
            else:
                assert False

### Egy példa

A jegyzetben megtalálható

#### A nyelv definiálása

In [8]:
# Terminálisok
terminals = {n:Terminal(n) for n in ['a', 'b', '+']}

# Nem terminálisok
non_terminals = {n:NonTerminal(n) for n in ['S', 'T']}

# Az összes szimbólum egy halmazban
symbols = {**terminals, **non_terminals}

# Segédfüggvények a szabályok kényelmesebb definiálásához
def parse_symbol_sequence(string):
    return [symbols[symbol] for symbol in string.split()]

def parse_rule(non_terminal, string):
    return (NonTerminal(non_terminal), parse_symbol_sequence(string))

S = non_terminals['S']

H = [parse_rule('S', 'T + S'),
     parse_rule('S', 'T'),
     parse_rule('T', 'a'),
     parse_rule('T', 'b'),
     ]

G = Lang(S, H)

#### Levezetés

Levezethetö-e a 'b + a' szó a fent definiált szabályokból?

In [9]:
LOGGING_ENABLED = False
word = 'b + a'
w = parse_symbol_sequence(word) + ['#']
deriveable, steps = G.is_deriveable(w)

if deriveable:
    print(f'A "{word}" szó levezethetö az {str(S)} nem-terminálisból következö behelyettesítések alkalmazásával:')
    display(Math(", ".join(str(step) for step in steps if isinstance(step, NonTerminal))))
else:
    print(f'A "{word}" szó nem vezethetö le az {str(S)} nem-terminálisból')

A "b + a" szó levezethetö az S nem-terminálisból következö behelyettesítések alkalmazásával:


<IPython.core.display.Math object>

A levezetés közben az algoritmus a következö lépéseket végezte:

In [10]:
LOGGING_ENABLED = True
deriveable, steps = G.is_deriveable(w)

<IPython.core.display.Math object>

Kiterjesztés


<IPython.core.display.Math object>

Kiterjesztés


<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

Sikeres input ellenörzés


<IPython.core.display.Math object>

Sikeres elemzés!


<IPython.core.display.Math object>